In [ ]:
import os
import glob
import numpy as np
import csv
import librosa

import matplotlib.pyplot as plt

In [ ]:
train_metadata_path = '../data/train.txt'
valid_metadata_path = '../data/valid.txt'

In [ ]:
train_file_list = list()
valid_file_list = list()

with open(train_metadata_path, 'r') as file:
    with open('local_train.txt', 'w') as csvfile:
        csv_writer = csv.writer(csvfile)

        while True:
            line = file.readline()

            if line is None or len(line) == 0:
                break

            contents = line.split(',')
            wav_file_path = contents[0]
            
            contents[2] = contents[2].strip()

            new_wav_file_path = os.path.join('train', wav_file_path.split('/')[-1])
            contents[0] = new_wav_file_path
            
            contents.pop(1)

            csv_writer.writerow(contents)

            
with open(valid_metadata_path, 'r') as file:
    with open('local_valid.txt', 'w') as csvfile:
        csv_writer = csv.writer(csvfile)

        while True:
            line = file.readline()

            if line is None or len(line) == 0:
                break

            contents = line.split(',')
            wav_file_path = contents[0]
            
            contents[2] = contents[2].strip()

            new_wav_file_path = os.path.join('valid', wav_file_path.split('/')[-1])
            contents[0] = new_wav_file_path
            
            contents.pop(1)

            csv_writer.writerow(contents)

In [ ]:
local_train_metadata_path = 'local_train.txt'
local_valid_metadata_path = 'local_valid.txt'

def load_class_statistics(metadata_path):
    
    class_counter_dict = dict()
    
    with open(metadata_path, 'r') as file:

        while True:
            line = file.readline()

            if line is None or len(line) == 0:
                break

            contents = line.split(',')
            label_string = contents[1].strip()

            labels = label_string.split('|')

            for label in labels:
                (start, end, class_name) = label.split('^')
                if class_name not in class_counter_dict.keys():
                    class_counter_dict[class_name] = 0
                class_counter_dict[class_name] += 1
                
    return class_counter_dict

In [ ]:
class_counter_train_dict = load_class_statistics(local_train_metadata_path)
class_counter_train_dict_list = sorted(class_counter_train_dict.items())
class_counter_train_dict = {pair[0]: pair[1] for pair in class_counter_train_dict_list}

class_counter_valid_dict = load_class_statistics(local_valid_metadata_path)
class_counter_valid_dict_list = sorted(class_counter_valid_dict.items())
class_counter_valid_dict = {pair[0]: pair[1] for pair in class_counter_valid_dict_list}


In [ ]:
plt.figure(figsize=(20, 4))
key_list = list(class_counter_train_dict.keys())

for i, key in enumerate(key_list):
    key_list[i] = key.replace('_', '\n').replace(' ', '\n')

fig = plt.bar(key_list, class_counter_train_dict.values(), 0.8, color='g')
fig = plt.bar(key_list, class_counter_valid_dict.values(), 0.8, color='r')

plt.show()

In [ ]:
os.makedirs('train', exist_ok=True)
os.makedirs('valid', exist_ok=True)

In [ ]:
key_list = list(class_counter_train_dict.keys())

for key in key_list:
    os.makedirs(os.path.join('train', key), exist_ok=True)
    os.makedirs(os.path.join('valid', key), exist_ok=True)

In [ ]:
local_train_metadata_path = 'local_train.txt'
local_valid_metadata_path = 'local_valid.txt'

def split_wav_file(metadata_path):
    
    with open(metadata_path, 'r') as file:

        while True:
            line = file.readline()

            if line is None or len(line) == 0:
                break

            contents = line.split(',')
            parent_path = contents[0].split('/')[0]
            wav_name = contents[0].split('/')[-1]
            
            data, fs = librosa.core.load(os.path.join('../data', contents[0]), sr=None)
            
#             print('[MAX: {:10f} MIN: {:10f}]'.format(max(data), min(data)))
            
            label_string = contents[1].strip()

            labels = label_string.split('|')
            
            print('[{}]'.format(contents[0]))
            print('({:10d} ~ {:10d})'.format(int(0), len(data)))

            for i, label in enumerate(labels):
                (start, end, class_name) = label.split('^')
                start_idx = int(float(start) * fs)
                end_idx = int(float(end) * fs)
                output_wav_name = wav_name.replace('.wav', '_{:02d}.wav'.format(i))
                output_file_name = os.path.join(os.path.join(parent_path, class_name), output_wav_name)
                
                librosa.output.write_wav(output_file_name, data[start_idx:end_idx], fs)
                print(output_file_name)
                print('[{:10d} ~ {:10d}] [{}]'.format(start_idx, end_idx, class_name))
                
            print()
                
    return

In [ ]:
split_wav_file(local_valid_metadata_path)